#### (1) K-means Algorithm

In [10]:
import cv2
import numpy as np

src = cv2.imread("../data/lena.png").astype(np.float32)/255

data = src.reshape(-1, 3)

criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)

retval, labels, centers = cv2.kmeans(data, 64, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = centers.clip(0,1)

dst = centers[labels].reshape(src.shape)

print('{} dB'.format(cv2.PSNR(src, dst,1)))

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

31.93308846886234 dB


- Lab Color Space

In [11]:
image = cv2.imread("../data/lena.png").astype(np.float32)/255
image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

data = image_lab.reshape(-1, 3)

criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)

retval, labels, centers = cv2.kmeans(data, 64, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

dst = centers[labels].reshape(image.shape)
recon = cv2.cvtColor(dst,cv2.COLOR_Lab2BGR)
recon = recon.clip(0,1)

print('{} dB'.format(cv2.PSNR(image, recon,1)))

cv2.imshow("reconstucted image", recon)
cv2.waitKey()
cv2.destroyAllWindows()

30.91326470853884 dB


- HSV Color Space

In [12]:
image = cv2.imread("../data/lena.png").astype(np.float32)/255
image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

data = image_hsv.reshape(-1, 3)

criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)

retval, labels, centers = cv2.kmeans(data, 64, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

dst = centers[labels].reshape(image.shape)
recon = cv2.cvtColor(dst,cv2.COLOR_HSV2BGR)
recon = recon.clip(0,1)

print('{} dB'.format(cv2.PSNR(image, recon,1)))

cv2.imshow("reconstucted image", recon)
cv2.waitKey()
cv2.destroyAllWindows()

22.144032688932764 dB


#### (2) Vector Quantization using K-Means Algorithm

In [24]:
import cv2
import numpy as np

src = cv2.imread("../data/lena.png").astype(np.float32)/255

pels = src.shape[0]*src.shape[1]
n = 8

data = src.reshape(-1, 3)
data = data.reshape((pels//n, n, 3))
data = data.reshape((pels//n,-1))

criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)

retval, labels, centers = cv2.kmeans(data, 64, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = centers.clip(0,1)

dst = centers[labels].reshape(-1,3)
dst = dst.reshape(image.shape)

k = 64*n*24 + (pels//n)*6
t = image.shape[0] * image.shape[1] * 24
print('압축율 : {:0.2f} : 1'.format(t/k))
print('비트레이트: {:0.2f} bpp'.format(k/pels))
print('복원화질: {} dB'.format(cv2.PSNR(src, dst,1)))

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

압축율 : 30.12 : 1
비트레이트: 0.80 bpp
복원화질: 27.032521445433492 dB


#### (3) Distance transform
- 화소마다 자신의 위치에서 배경 (0 레벨인 화소 집합) 까지의 거리를 구한다.
- 배경에 포함된 화소의 거리는 0으로 한다.
- 오브젝트 내에서 이 거리가 가장 먼 점과 가장 가까운 점을 어떻게 활용할 수 있을까?

In [26]:
import cv2
import numpy as np

image = np.full((480, 640), 255, np.uint8)
image = cv2.rectangle(image, (200, 0), (500, 480), 0, 20)

distmap = cv2.distanceTransform(image, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
print(distmap.shape, distmap.dtype, distmap.min(), distmap.max())

cv2.imshow('image', image)
cv2.imshow('distancemap', cv2.normalize(distmap,None,0,1,cv2.NORM_MINMAX))
cv2.waitKey()
cv2.destroyAllWindows()

(480, 640) float32 0.0 190.0


#### (4) Watershed Algorithm

In [27]:
import numpy as np
import cv2 as cv

img = cv.imread('../data/water_coins.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
blur = cv.GaussianBlur(gray, (7,7), 0)
_, mask = cv.threshold(blur,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(mask,cv.MORPH_OPEN,kernel, iterations = 2)

# 강제 확장된 foreground
ext_fg = cv.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
# 배경과의 거리가 최대값의 80% 이상인 화소들만 남긴다
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
_, sure_fg = cv.threshold(dist_transform,0.8*dist_transform.max(),255,cv2.THRESH_BINARY)

# Finding unknown region
# 영역 확장에서 sure_fg를 제외한 영역
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(ext_fg, sure_fg)

# Marker labelling of sure_fg
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1
# Now, mark the region of unknown with zero
# marker for the sure_bg is 1
# markers for objects are 2..markers.max()
markers[unknown==255] = 0
# marker가 0인 화소들의 marker를 새로 할당한다.
# 경계선에 존재하는 화소들은 marker 값을 -1로 한다.
markers = cv.watershed(img, markers)
img[markers == -1] = [0,255,255]

cv2.imshow('images with boundaries', img)
cv2.imshow('markers', (markers*15).astype(np.uint8))
cv2.waitKey()
cv2.destroyAllWindows()